In [2]:
import os
import json
import requests
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

load_dotenv(override=True)
openai = OpenAI()
anthropic = Anthropic() 

this is a test

In [9]:
# Load the CSV file correctly - skip the first line and use line 2 as headers
df = pd.read_csv('ThePauperCube.csv', skiprows=1)

print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())
print(f"\nDataset info:")
df.info()

Dataset shape: (450, 8)

Column names:
['name', 'CMC', 'Type', 'Color', 'Color Category', 'Oracle Text', 'tags', 'MTGO ID']

First few rows:
                 name  CMC                      Type Color Color Category  \
0         Boros Elite    1  Creature - Human Soldier     W          White   
1     Deftblade Elite    1  Creature - Human Soldier     W          White   
2     Doomed Traveler    1  Creature - Human Soldier     W          White   
3      Elite Vanguard    1  Creature - Human Soldier     W          White   
4  Faerie Guidemother    1        Creature - Faerie      W          White   

   Oracle Text                                            tags   MTGO ID  
0          NaN                                             NaN  120547.0  
1          NaN                                             NaN   18617.0  
2          NaN  GTC Update;token generator;WR tokens;WG tokens   42650.0  
3          NaN                                      EMA Update   60565.0  
4          NaN       

In [32]:
import time

# Loop through and get all MTGO IDs, then fetch card details from Scryfall
print("Extracting MTGO IDs and fetching card details...")
print("=" * 50)

# Create a list to store oracle texts
oracle_texts = []

# Loop through each row
for index, row in df.iterrows():
    mtgo_id = row['MTGO ID']
    card_name = row['name']
    
    if pd.notna(mtgo_id):
        try:
            # Convert to int to avoid decimal issues
            mtgo_id_int = int(mtgo_id)
            url = f"https://api.scryfall.com/cards/mtgo/{mtgo_id_int}"
            
            print(f"Fetching data for: {card_name} (MTGO ID: {mtgo_id_int})")
            
            # Make API request
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for bad status codes
            
            card_data = response.json()
            
            # Extract oracle text
            oracle_text = None
            
            # Check if it's a double-faced card
            if "card_faces" in card_data and card_data["card_faces"]:
                # For double-faced cards, combine oracle text from both faces
                oracle_texts_faces = []
                for face in card_data["card_faces"]:
                    if "oracle_text" in face and face["oracle_text"]:
                        oracle_texts_faces.append(face["oracle_text"])
                oracle_text = " // ".join(oracle_texts_faces) if oracle_texts_faces else None
            else:
                # For single-faced cards
                oracle_text = card_data.get("oracle_text")
            
            # Update the DataFrame
            df.at[index, 'Oracle Text'] = oracle_text
            
            print(f"Oracle Text: {oracle_text[:100]}..." if oracle_text and len(oracle_text) > 100 else f"Oracle Text: {oracle_text}")
            print("-" * 50)
            
            # Add a small delay to be respectful to the API
            time.sleep(0.1)
            
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {card_name} (MTGO ID: {mtgo_id_int}): {e}")
            df.at[index, 'Oracle Text'] = None
        except Exception as e:
            print(f"Unexpected error for {card_name}: {e}")
            df.at[index, 'Oracle Text'] = None
    else:
        print(f"No MTGO ID for: {card_name}")
        df.at[index, 'Oracle Text'] = None

print("\nFinished updating Oracle Text for all cards!")
print(f"Cards with Oracle Text: {df['Oracle Text'].notna().sum()}")
print(f"Cards without Oracle Text: {df['Oracle Text'].isna().sum()}")

Extracting MTGO IDs and fetching card details...
Fetching data for: Boros Elite (MTGO ID: 120547)


/var/folders/lf/f83j_kbn0qdbdd7j8rwyj0m80000gn/T/ipykernel_75554/868512965.py:45: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Battalion — Whenever this creature and at least two other creatures attack, this creature gets +2/+2 until end of turn.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[index, 'Oracle Text'] = oracle_text


Oracle Text: Battalion — Whenever this creature and at least two other creatures attack, this creature gets +2/+2...
--------------------------------------------------
Fetching data for: Deftblade Elite (MTGO ID: 18617)
Oracle Text: Provoke (Whenever this creature attacks, you may have target creature defending player controls unta...
--------------------------------------------------
Fetching data for: Doomed Traveler (MTGO ID: 42650)
Oracle Text: When this creature dies, create a 1/1 white Spirit creature token with flying.
--------------------------------------------------
Fetching data for: Elite Vanguard (MTGO ID: 60565)
Oracle Text: 
--------------------------------------------------
Fetching data for: Faerie Guidemother (MTGO ID: 78110)
Oracle Text: Flying // Target creature gets +2/+1 and gains flying until end of turn. (Then exile this card. You ...
--------------------------------------------------
Fetching data for: Gideon's Lawkeeper (MTGO ID: 41529)
Oracle Text: {W}, {T}: 

In [ ]:
# Display the updated DataFrame with Oracle Text
print("Updated DataFrame with Oracle Text:")
print("=" * 50)

# Show cards that now have Oracle Text
cards_with_oracle = df[df['Oracle Text'].notna()]
print(f"\nCards with Oracle Text ({len(cards_with_oracle)}):")
for index, row in cards_with_oracle.head(10).iterrows():
    oracle_preview = row['Oracle Text'][:80] + "..." if len(str(row['Oracle Text'])) > 80 else row['Oracle Text']
    print(f"{row['name']}: {oracle_preview}")

if len(cards_with_oracle) > 10:
    print(f"... and {len(cards_with_oracle) - 10} more cards")

# Show updated DataFrame info
print(f"\nDataset info after updates:")
df.info()

# Save the updated DataFrame to a new CSV file
output_filename = 'jumpstart_updated.csv'
df.to_csv(output_filename, index=False)
print(f"\nUpdated data saved to: {output_filename}")

Updated DataFrame with Oracle Text:

Cards with Oracle Text (424):
Boros Elite: Battalion — Whenever this creature and at least two other creatures attack, this...
Deftblade Elite: Provoke (Whenever this creature attacks, you may have target creature defending ...
Doomed Traveler: When this creature dies, create a 1/1 white Spirit creature token with flying.
Elite Vanguard: 
Faerie Guidemother: Flying // Target creature gets +2/+1 and gains flying until end of turn. (Then e...
Gideon's Lawkeeper: {W}, {T}: Tap target creature.
Goldmeadow Harrier: {W}, {T}: Tap target creature.
Miner's Guidewing: Flying, vigilance
When this creature dies, target creature you control explores....
Novice Inspector: When this creature enters, investigate. (Create a Clue token. It's an artifact w...
Savannah Lions: 
... and 414 more cards

Dataset info after updates:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 8 columns):
 #   Column          Non-Null Count  D

In [35]:
# Fix MTGO ID data type issue and save the corrected file
print("Fixing MTGO ID data types...")
print("=" * 40)

# Create a copy of the DataFrame to avoid modifying the original
df_corrected = df.copy()

# Convert MTGO ID to proper integer format, handling NaN values
def fix_mtgo_id(value):
    if pd.isna(value):
        return ""  # Return empty string for missing values
    else:
        return str(int(value))  # Convert to int then to string to remove .0

df_corrected['MTGO ID'] = df_corrected['MTGO ID'].apply(fix_mtgo_id)

print("MTGO ID data type after conversion:")
print(df_corrected['MTGO ID'].dtype)
print(f"Sample MTGO IDs: {df_corrected[df_corrected['MTGO ID'] != '']['MTGO ID'].head().tolist()}")

# Save the corrected DataFrame to jumpstart_updated.csv
output_filename = 'jumpstart_updated.csv'
df_corrected.to_csv(output_filename, index=False)
print(f"\nCorrected data saved to: {output_filename}")

# Verify the saved file doesn't have .0 extensions
print("\nVerifying saved file (first few lines):")
with open(output_filename, 'r') as f:
    for i, line in enumerate(f):
        print(f"Line {i}: {line.strip()}")
        if i >= 4:  # Show first 5 lines
            break

Fixing MTGO ID data types...
MTGO ID data type after conversion:
object
Sample MTGO IDs: ['120547', '18617', '42650', '60565', '78110']

Corrected data saved to: jumpstart_updated.csv

Verifying saved file (first few lines):
Line 0: name,CMC,Type,Color,Color Category,Oracle Text,tags,MTGO ID
Line 1: Boros Elite,1,Creature - Human Soldier,W,White,"Battalion — Whenever this creature and at least two other creatures attack, this creature gets +2/+2 until end of turn.",,120547
Line 2: Deftblade Elite,1,Creature - Human Soldier,W,White,"Provoke (Whenever this creature attacks, you may have target creature defending player controls untap and block it if able.)
Line 3: {1}{W}: Prevent all combat damage that would be dealt to and dealt by this creature this turn.",,18617
Line 4: Doomed Traveler,1,Creature - Human Soldier,W,White,"When this creature dies, create a 1/1 white Spirit creature token with flying.",GTC Update;token generator;WR tokens;WG tokens,42650


In [37]:
# Fix newlines in Oracle Text for proper CSV formatting
print("Fixing newlines in Oracle Text...")
print("=" * 50)

# Create a new corrected DataFrame
df_final = df_corrected.copy()

# Function to clean Oracle Text by replacing newlines
def clean_oracle_text(text):
    if pd.isna(text) or text == "":
        return text
    # Replace newlines with " | " which is a common delimiter for multi-line text in CSV
    cleaned_text = str(text).replace('\n', ' | ').replace('\r', ' | ')
    return cleaned_text

# Apply the cleaning function to Oracle Text column
df_final['Oracle Text'] = df_final['Oracle Text'].apply(clean_oracle_text)

# Show some examples of the cleaned Oracle Text
print("Examples of cleaned Oracle Text:")
print("-" * 30)

cards_with_oracle = df_final[df_final['Oracle Text'].notna() & (df_final['Oracle Text'] != "")]
for i, (index, row) in enumerate(cards_with_oracle.head(5).iterrows()):
    oracle_text = row['Oracle Text']
    if '|' in oracle_text:  # Show examples that had newlines
        print(f"Card: {row['name']}")
        print(f"Oracle: {oracle_text}")
        print("-" * 30)
        break

# Save the final corrected CSV with proper quoting to handle commas and quotes
final_filename = 'jumpstart_updated_clean.csv'
df_final.to_csv(final_filename, index=False, quoting=1)  # quoting=1 means QUOTE_ALL
print(f"\nFinal cleaned data saved to: {final_filename}")

# Verify the file format
print(f"\nVerifying {final_filename} (first few lines):")
with open(final_filename, 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines[:5]):
        print(f"Line {i}: {line.strip()}")
        
print(f"\nTotal lines in file: {len(lines)}")
print(f"Expected lines: {len(df_final) + 1} (data + header)")

# Read the file back to verify it loads correctly
try:
    test_df = pd.read_csv(final_filename)
    print(f"✅ File loads correctly with {len(test_df)} rows and {len(test_df.columns)} columns")
    print(f"Column names: {test_df.columns.tolist()}")
    
    # Check for any Oracle Text entries that still have the | separator
    oracle_with_separator = test_df[test_df['Oracle Text'].str.contains('|', na=False)]
    print(f"Cards with multi-line Oracle Text (now using |): {len(oracle_with_separator)}")
    
except Exception as e:
    print(f"❌ Error reading the file: {e}")

Fixing newlines in Oracle Text...
Examples of cleaned Oracle Text:
------------------------------
Card: Deftblade Elite
Oracle: Provoke (Whenever this creature attacks, you may have target creature defending player controls untap and block it if able.) | {1}{W}: Prevent all combat damage that would be dealt to and dealt by this creature this turn.
------------------------------

Final cleaned data saved to: jumpstart_updated_clean.csv

Verifying jumpstart_updated_clean.csv (first few lines):
Line 0: "name","CMC","Type","Color","Color Category","Oracle Text","tags","MTGO ID"
Line 1: "Boros Elite","1","Creature - Human Soldier","W","White","Battalion — Whenever this creature and at least two other creatures attack, this creature gets +2/+2 until end of turn.","","120547"
Line 2: "Deftblade Elite","1","Creature - Human Soldier","W","White","Provoke (Whenever this creature attacks, you may have target creature defending player controls untap and block it if able.) | {1}{W}: Prevent all com